In [9]:
from xbbg import blp
import pandas as pd
from datetime import timedelta
from IPython.display import clear_output
import time

In [10]:
banks = pd.read_csv('Bank_Secondaries.csv')
banks.columns = banks.columns.str.strip()
banks = banks.drop_duplicates().iloc[:-1]
banks['Announced Date'] = pd.to_datetime(banks['Announced Date'])
banks = banks[(banks['Announced Date'] >= '2014-01-01') & (banks['Announced Date'] < '2024-01-01')]

In [11]:
banks

,Announced Date,Issuer Ticker,Issuer Name,Offer Size (M),Offer Type,Offer Stage,Offer Price,Last Close,Offer To Date,Last Price,...,Instit Owner (Shares Held),Primary Exchange,Primary Shares (M),Secondary Shares (M),Country/Region,Retail Shares (M),Left Lead Bookrunner,Instit Owner (# of Buyers),mktcap,Factor
3,2023-12-06,5838 JP,Rakuten Bank Ltd,411.9170,"ADDL,Secondary Share Offering,PE Backed,PE Exit",Trading,2470.00,3285.000,32.995953,3285.000,...,5.660000e+07,Tokyo,NaN,24.549999,JN,NaN,NaN,28.0,"3,245.02",0.13
4,2023-12-05,8600 JP,TOMONY Holdings Inc,74.1437,"ADDL,Primary Share Offering",Trading,366.00,408.000,11.475410,408.000,...,7.440000e+07,Tokyo,28.0000,NaN,JN,NaN,NaN,23.0,530.81,0.14
5,2023-12-04,BARC LN,Barclays PLC,643.7180,"ADDL,Secondary Share Offering,Accelerated Book...",Trading,141.00,178.480,29.886524,178.480,...,7.990000e+09,London,NaN,361.700012,GB,NaN,NaN,207.0,"27,313.31",0.02
6,2023-11-20,SOR NO,Sparebanken Sor,245.1960,"ADDL,Secondary Share Offering,REG S,Rule 144A",Trading,125.00,142.800,14.400000,142.800,...,2.170000e+07,Oslo,NaN,21.001801,NO,NaN,NaN,3.0,482.89,0.51
7,2023-11-06,AIBG ID,AIB Group PLC,552.7210,"ADDL,Secondary Share Offering,Accelerated Book...",Trading,3.93,4.876,24.783712,4.876,...,2.180000e+09,EN Dublin,NaN,131.000000,IR,NaN,NaN,169.0,"11,414.68",0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2014-02-27,BKIA SM,Bankia SA,1788.3800,"ADDL,Secondary Share Offering,Accelerated Book...",Trading,1.51,NaN,NaN,NaN,...,2.060000e+09,,NaN,863.799988,SP,NaN,NaN,NaN,"6,237.61",0.29
403,2014-01-28,BKU US,BankUnited Inc,291.1800,"ADDL,Secondary Share Offering,PE Backed,PE Exi...",Trading,31.65,24.940,-21.200632,24.595,...,8.060000e+07,New York,NaN,8.000000,US,NaN,NaN,94.0,"3,220.58",0.09
404,2014-01-22,TCBI US,Texas Capital Bancshares Inc,112.9690,"ADDL,Primary Share Offering",Trading,60.25,56.430,-6.340249,55.785,...,5.230000e+07,NASDAQ GS,1.7000,NaN,US,NaN,Deutsche Bank,112.0,"2,603.18",0.04
405,2014-01-21,RBI AV,Raiffeisen Bank International AG,3764.2000,"ADDL,Primary Share Offering,Accelerated Bookbuild",Trading,28.50,17.120,-35.513380,17.120,...,2.210000e+08,Vienna,97.4739,NaN,AS,NaN,Deutsche Bank,49.0,"8,271.24",0.46


In [12]:
# Convert the 'Announced Date' column to datetime format if it's not already
banks['Announced Date'] = pd.to_datetime(banks['Announced Date'])

# Sort the dataframe by 'Ticker' and 'Announced Date' to ensure chronological order
banks_sorted = banks.sort_values(['Issuer Ticker', 'Announced Date'])

# Group by 'Ticker' and filter out close dates
def filter_dates(group):
    group = group[group['Announced Date'].diff().fillna(pd.Timedelta(days=31)) > pd.Timedelta(days=30)]
    return group

# Apply the filtering function
filtered_banks = banks_sorted.groupby('Issuer Ticker', as_index=False).apply(filter_dates).reset_index(drop=True)

In [13]:
#banks = filtered_banks.copy()
banks[banks.columns[3]] = pd.to_numeric(banks[banks.columns[3]], errors='coerce')
banks['mktcap'] = pd.to_numeric(banks['mktcap'], errors='coerce')

# Calculate 'Factor'
banks["Factor"] = banks[banks.columns[3]] / banks["mktcap"]
banks['Factor'].fillna(banks['Factor'].median(), inplace=True)

banks[['Announced Date', "Issuer Ticker", "Factor"]] \
    .rename(columns={'Announced Date': 'Date', 'Issuer Ticker': "Ticker", "Factor": "Factor"}) \
    .reset_index(drop=True) \
    .to_csv("signal-raw_all.csv",index=False)

In [ ]:
banks

In [ ]:
final_prices_df = pd.DataFrame()
empty_df_tickers = []

for index, row in banks.iterrows():
    ticker = row['Issuer Ticker']
    date = row['Announced Date']
    end_date = date + timedelta(days=30)
    
    date_str = f"{date.year}-{date.month}-{date.day}"
    end_date_str = f"{end_date.year}-{end_date.month}-{end_date.day}"
    test_df = blp.bdh(f'{ticker} EQUITY',flds= ["PX_LAST"], start_date=date_str, end_date=end_date_str,FX="USD",CshAdjNormal=True, CshAdjAbnormal=True)
    
    if test_df.empty:
        empty_df_tickers.append(ticker)
    else:
        test_df = test_df.reset_index()
        test_df['Ticker'] = ticker
        test_df.columns = ['Date','Price','Ticker']
        final_prices_df = pd.concat([final_prices_df,test_df])
    print(date)
    print(ticker)
    clear_output(wait=True)

In [ ]:
empty_df_tickers

In [ ]:
final_prices_df.to_csv('bbg_data.csv')

In [ ]:
pd.read_csv(('bbg_data.csv')).drop_duplicates()

In [ ]:
pd.read_csv(('bbg_data.csv')).drop_duplicates().iloc[:,2:].to_csv("bbg_data.csv")

In [ ]:
final_prices_df['Date'] = pd.to_datetime(final_prices_df['Date'])

# Sort the DataFrame by 'ticker' and 'Date'
final_prices_df.sort_values(by=['Ticker', 'Date'], inplace=True)

# Calculate the percentage change in 'Price' for each 'ticker'
final_prices_df['ret'] = final_prices_df.groupby('Ticker')['Price'].pct_change()

final_prices_df['ret'].fillna(0, inplace=True)

final_prices_df.rename(columns={'Price': 'Adj Close'}, inplace=True)
final_prices_df = final_prices_df[["Date","Ticker","Adj Close","ret"]]

In [ ]:
final_prices_df.to_csv('bbg_data.csv')

In [ ]:
banks[banks.columns[3]] = pd.to_numeric(banks[banks.columns[3]], errors='coerce')
banks['mktcap'] = pd.to_numeric(banks['mktcap'], errors='coerce')

# Calculate 'Factor'
banks["Factor"] = banks[banks.columns[3]] / banks["mktcap"]
banks['Factor'].fillna(banks['Factor'].median(), inplace=True)

banks[['Announced Date', "Issuer Ticker", "Factor"]] \
    .rename(columns={'Announced Date': 'Date', 'Issuer Ticker': "Ticker", "Factor": "Factor"}) \
    .reset_index(drop=True) \
    .to_csv("signal-raw_all.csv",index=False)

In [ ]:
banks

In [ ]:
banks[['Announced Date', "Issuer Ticker", "Factor"]] \
    .rename(columns={'Announced Date': 'Date', 'Issuer Ticker': "Ticker", "Factor": "Factor"}) \
    .reset_index(drop=True)

In [ ]:
pd.read_csv("bitcoin.csv")

In [ ]:
final_prices_df = pd.concat([final_prices_df,pd.read_csv("bitcoin.csv")])
final_prices_df.to_csv('bbg_data.csv')

In [ ]:
sdf = pd.read_csv('signal-raw_all.csv')

In [ ]:
sdf[sdf['Date'] == datetime.datetime]

In [ ]:
tdf = pd.read_csv("bbg_data.csv")

In [ ]:
tdf.iloc[:,1:].to_csv("bbg_data.csv",index=False)

In [ ]:
tdf.dropna().to_csv("bbg_data.csv",index=False)

In [ ]:
tdf['Date']= pd.to_datetime(tdf['Date'])